In [2]:
import sys
sys.path.append('../..')
import os

from tqdm import tqdm
import datetime

import pandas as pd
import numpy as np

from src.data.STAR.star import STARDataset

if not os.path.exists('star_results'):
    os.makedirs('star_results')

path_star_dataset = "../../src/data/STAR/STAR_Students.csv"

In [3]:
from sklearn.linear_model import (
    LogisticRegressionCV,
    RidgeCV,
)
from sklearn.ensemble import HistGradientBoostingRegressor
from src.randomization_aware.combine_cate import CATECombiner
from src.randomization_aware.learners import (
    DRLearner,
    QuasiOptimizedLearner,
)
from src.baselines.asaiee import AsaieeCATE
from src.baselines.ksp import KSPCATE
from src.baselines.pooling import TLearnerPooling
from src.baselines.trial_only import TrialCATE
from src.baselines.predict_ate import DifferenceInMeans
from econml.metalearners import TLearner
from econml.dr import DRLearner as econml_DRLearner

crossfit_folds = 2

regressor_outcome = HistGradientBoostingRegressor
alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
regressor_cate = lambda: RidgeCV(alphas=alphas)
study_classifier = lambda: LogisticRegressionCV(
    max_iter=1000, Cs=[1 / a for a in alphas], cv=3, solver="liblinear"
)

cate_estimator_tlearner = lambda: TLearner(models=regressor_outcome())
cate_estimator_dr = lambda: econml_DRLearner(
    model_propensity=study_classifier(),
    model_regression=regressor_outcome(),
    model_final=regressor_cate(),
    cv=crossfit_folds,
)


def get_drlearner_star(propensity_score):
    return DRLearner(
        propensity_score=propensity_score,
        regressor_cate=regressor_cate(),
        regressor_control=regressor_outcome(),
        regressor_treated=regressor_outcome(),
        crossfit_folds=crossfit_folds,
    )


def get_quasioptimized_star(propensity_score):
    return QuasiOptimizedLearner(
        propensity_score=propensity_score,
        regressor_cate=regressor_cate(),
        regressor_control=regressor_outcome(),
        regressor_treated=regressor_outcome(),
        study_classifier=study_classifier(),
        crossfit_folds=crossfit_folds,
    )


def get_quasioptimized_star_unweighted(propensity_score):
    return QuasiOptimizedLearner(
        propensity_score=propensity_score,
        regressor_cate=regressor_cate(),
        regressor_control=regressor_outcome(),
        regressor_treated=regressor_outcome(),
        study_classifier=study_classifier(),
        crossfit_folds=crossfit_folds,
        remove_study_weighting=True,
    )


def get_combined_star(propensity_score):
    return CATECombiner(
        propensity_score=propensity_score,
        cate_learner_1=get_drlearner_star(propensity_score),
        cate_learner_2=get_quasioptimized_star(propensity_score),
    )


def get_asaiee_star(propensity_score):
    return AsaieeCATE(
        propensity_score=propensity_score,
        regressor_cate=regressor_cate(),
        regressor_control=regressor_outcome(),
        regressor_treated=regressor_outcome(),
        crossfit_folds=crossfit_folds,
    )


def get_ksp_star(propensity_score):
    return KSPCATE(
        propensity_score,
        cate_estimator=cate_estimator_tlearner(),
        bias_correction_model=regressor_outcome(),
    )


def get_pooling_star(propensity_score=None):
    return TLearnerPooling(
        regressor_control=regressor_outcome(),
        regressor_treated=regressor_outcome(),
        study_classifier=study_classifier(),
    )


def get_tlearner_star(propensity_score=None):
    return TrialCATE(cate_estimator=cate_estimator_tlearner())


def get_dm(propensity_score=None):
    return DifferenceInMeans()

In [4]:
def get_max_sample_sizes_independent(rct_fraction_of_rural, eval_fraction_of_rct):
    dgp = STARDataset(path_star_dataset)
    
    max_n1 = 0
    for n1 in range(1, 10000, 10):  # Increment n1 in steps of 100
        try:
            dgp.sample(n1, 1, rct_fraction_of_rural=rct_fraction_of_rural, eval_fraction_of_rct=eval_fraction_of_rct)
            max_n1 = n1
        except Exception as e:
            print(f"Error for n1={n1}: {e}")
            break  # Stop increasing n1 if an error occurs

    max_n0 = 0
    for n0 in range(1, 10000, 10):  # Increment n0 in steps of 100
        try:
            dgp.sample(1, n0, rct_fraction_of_rural=rct_fraction_of_rural, eval_fraction_of_rct=eval_fraction_of_rct)
            max_n0 = n0
        except Exception as e:
            print(f"Error for n0={n0}: {e}")
            break  # Stop increasing n0 if an error occurs

    return max_n1, max_n0
    

rct_fraction_of_rural=0.5
eval_fraction_of_rct=0.2
get_max_sample_sizes_independent(rct_fraction_of_rural, eval_fraction_of_rct)


Error for n1=1131: Cannot take a larger sample than population when 'replace=False'
Error for n0=2221: Cannot take a larger sample than population when 'replace=False'


(1121, 2211)

In [5]:
methods = {
    "DR-learner": get_drlearner_star,
    "QR-learner": get_quasioptimized_star,
    # "QR-learner (unweighted)" : get_quasioptimized_star_unweighted,
    "Combined QR- and DR-learner": get_combined_star,
    "Asaiee et al. [2023]": get_asaiee_star,
    "Kallus et al. [2018]": get_ksp_star,
    "Pooled T-learner": get_pooling_star,
    "T-Learner": get_tlearner_star,
    "Predict ATE": get_dm,
}

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"timestamp: {timestamp}")
iterations = 1
n1_list = [1000]
n0_list = [100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000]
fraction_rural_list = [0.5]
covar_to_drop = [
    "g1surban",
]
rows = []

full_covar_list = [
    "g1surban",
    "gender",
    "race",
    "birthmonth",
    "birthday",
    "birthyear",
    "gkfreelunch",
    "g1tchid",
    "g1freelunch",
]

for dropped_covar in covar_to_drop:
    if dropped_covar is None:
        covar_list = full_covar_list
    else:
        if isinstance(dropped_covar, list):
            if all(covar in full_covar_list for covar in dropped_covar):
                covar_list = [
                    covar for covar in full_covar_list if covar not in dropped_covar
                ]
            else:
                missing = [
                    covar for covar in dropped_covar if covar not in full_covar_list
                ]
                raise ValueError(f"{missing} not found in full_covar_list")
        elif dropped_covar in full_covar_list:
            covar_list = [covar for covar in full_covar_list if covar != dropped_covar]
        else:
            raise ValueError(f"{dropped_covar} not found in full_covar_list")
    dgp = STARDataset(path_star_dataset, cat_covar_columns=covar_list)
    propensity_score_rct = dgp.get_propensity_score()
    for fraction_rural in fraction_rural_list:
        for n1 in n1_list:
            for n0 in n0_list:
                print(f"dropped_covar = {dropped_covar}, n1 = {n1}; n0 = {n0}")
                for i in tqdm(range(iterations)):

                    X_train, S_train, A_train, Y_train, X_eval, gt_adjusted_ite_eval = (
                        dgp.sample(
                            n1,
                            n0,
                            rct_fraction_of_rural=fraction_rural,
                            eval_fraction_of_rct=eval_fraction_of_rct,
                        )
                    )

                    for method_name, method_func in methods.items():

                        estimator = method_func(propensity_score_rct)

                        try:
                            estimator.fit(X_train, S_train, A_train, Y_train)
                            predictions = estimator.predict(X_eval)

                            assert predictions.shape == gt_adjusted_ite_eval.shape
                            rmse = np.sqrt(
                                np.mean((gt_adjusted_ite_eval - predictions) ** 2)
                            )
                            rows.append(
                                {
                                    "i": i,
                                    "n1": n1,
                                    "n0": n0,
                                    "dropped_covar": (
                                        dropped_covar
                                        if dropped_covar is not None
                                        else "None dropped"
                                    ),
                                    "fraction_rural": fraction_rural,
                                    "method": method_name,
                                    "rmse": rmse
                                }
                            )
                        except Exception as e:
                            rows.append(
                                {
                                    "i": i,
                                    "n1": n1,
                                    "n0": n0,
                                    "dropped_covar": (
                                        dropped_covar
                                        if dropped_covar is not None
                                        else "None dropped"
                                    ),
                                    "method": method_name,
                                    "error": str(e),
                                }
                            )

                        # Save results in a CSV file with a timestamp
                        results_df = pd.DataFrame(rows)
                        results_df.to_csv(
                            f"star_results/experiment_{timestamp}.csv", index=False
                        )

timestamp: 20250519_104710
dropped_covar = g1surban, n1 = 1000; n0 = 100


100%|██████████| 1/1 [00:22<00:00, 22.04s/it]


dropped_covar = g1surban, n1 = 1000; n0 = 250


100%|██████████| 1/1 [00:23<00:00, 23.06s/it]


dropped_covar = g1surban, n1 = 1000; n0 = 500


...

100%|██████████| 1/1 [00:22<00:00, 22.58s/it]


dropped_covar = g1surban, n1 = 1000; n0 = 1750


100%|██████████| 1/1 [00:30<00:00, 30.28s/it]


dropped_covar = g1surban, n1 = 1000; n0 = 2000


100%|██████████| 1/1 [00:29<00:00, 29.98s/it]


In [6]:
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler


dgp = STARDataset(
    path_star_dataset,
    cat_covar_columns=[
        "g1surban",
        "gender",
        "race",
        "birthmonth",
        "birthday",
        "birthyear",
        "gkfreelunch",
        "g1tchid",
        "g1freelunch",
    ],
)

iterations = 1000
reject_treated_list = []
reject_control_list = []

for i in tqdm(range(iterations)):


    X_train, S_train, A_train, Y_train, X_eval, gt_adjusted_ite_eval = dgp.sample(
        1000,
        1000,
        rct_fraction_of_rural=0.5,
        eval_fraction_of_rct=0.2,
    )

    # Standardize X_train
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Fit a linear regression model for S_train on X_train
    reg = LinearRegression().fit(X_train_scaled, S_train)

    # Separate data based on A_train
    mask_treated = A_train.flatten() == 1
    mask_control = A_train.flatten() == 0

    # Get residuals of S_train for treated and control groups
    S_train_residuals_treated = S_train[mask_treated] - reg.predict(X_train_scaled[mask_treated])
    S_train_residuals_control = S_train[mask_control] - reg.predict(X_train_scaled[mask_control])

    # Fit a linear regression model for Y_train on X_train for treated and control groups
    reg_treated = LinearRegression().fit(X_train_scaled[mask_treated], Y_train[mask_treated])
    reg_control = LinearRegression().fit(X_train_scaled[mask_control], Y_train[mask_control])

    # Get residuals of Y_train for treated and control groups
    Y_train_residuals_treated = Y_train[mask_treated] - reg_treated.predict(X_train_scaled[mask_treated])
    Y_train_residuals_control = Y_train[mask_control] - reg_control.predict(X_train_scaled[mask_control])

    # Compute the partial correlation for treated and control groups
    corr_treated, p_value_treated = pearsonr(
        Y_train_residuals_treated.flatten(), S_train_residuals_treated.flatten()
    )
    corr_control, p_value_control = pearsonr(
        Y_train_residuals_control.flatten(), S_train_residuals_control.flatten()
    )

    # Count number of rejections
    reject_treated = p_value_treated < 0.05
    reject_control = p_value_control < 0.05

    reject_treated_list.append(reject_treated)
    reject_control_list.append(reject_control)

print("Rejection rate of null for treated (transportability holds):", np.mean(reject_treated_list))
print("Rejection rate of null for control (transportability holds):", np.mean(reject_control_list))

100%|██████████| 1000/1000 [02:22<00:00,  7.00it/s]

Rejection rate of null for treated (transportability holds): 0.998
Rejection rate of null for control (transportability holds): 0.073
